In [65]:
from astropy.table import Table
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import h5py
import matplotlib.pyplot as plt
from gaiaxpy import generate, PhotometricSystem
import pandas as pd

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 110)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
# defining the Dataset class
class train_set(Dataset):
    def __init__(self,file):
        fn = h5py.File(file, 'r')
        self.f = fn
        
    def __len__(self):
        return self.f['group_1']['data'].shape[1]
  
    def __getitem__(self, index):
        # get data
        dset = self.f['group_1']['data']
        x = dset[:].T
        x = torch.Tensor(x[index])
        

        # get label
        ydset = self.f['group_1']['label']
        y = ydset[:].T
        y = torch.Tensor(y[index])
        # torch.from_numpy(y[index]) does not work since y is doubles and not floats.
        
        # get error in label # comment out for non-error label runs
        errdset = self.f['group_1']['e_label']
        err = errdset[:].T
        err = torch.Tensor(err[index])
        return (x,y,err)

class test_set(Dataset):
    def __init__(self,file):
        fn = h5py.File(file, 'r')
        self.f = fn
        
    def __len__(self):
        return self.f['group_2']['data'].shape[1]
  
    def __getitem__(self, index):
        # get data
        dset = self.f['group_2']['data']
        x = dset[:].T
        x = torch.from_numpy(x[index])

        # get label
        ydset = self.f['group_2']['label']
        y = ydset[:].T
        y = torch.from_numpy(y[index])
        
        # get error in label # comment out for non-error label runs
        errdset = self.f['group_2']['e_label']
        err = errdset[:].T
        err = torch.from_numpy(err[index])
        return (x.float(),y.float(),err.float())

In [3]:
# training_data = train_set("/arc/home/aydanmckay/mydataelabelssmallscalecuts.h5")
data = test_set("/arc/home/aydanmckay/mydataelabelssmallscalecuts.h5")
loaded_data = DataLoader(
    data,
    batch_size=32,
    # shuffle=True
)

In [4]:
model = Net()
model.load_state_dict(torch.load("/arc/home/aydanmckay/torchmodel/torchmodelWL1smallscalecutsep5.pth"))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=110, bias=True)
  )
)

In [139]:
preds = []
# covbp = np.zeros((55,55))
# covrp = np.zeros((55,55))
covbs = []
covrs = []
with torch.no_grad():
    for X, y, z in loaded_data:
        pred = model(X)
        for prediction,err in zip(pred,z):
            covbp = np.zeros((55,55))
            covrp = np.zeros((55,55))
            preds.append(prediction)
            for it in range(len(err[:55])):
                covbp[it][it] += err[it].item()**2
                covrp[it][it] += err[it+55].item()**2
            covbs.append(covbp)
            covrs.append(covrp)
        # preds.append(pred)
        # for m in z:
        #     for i in range(len(m[:55])):
        #         covbp[i][i] += m[i].item()**2
        #         covrp[i][i] += m[55+i].item()**2
        #     break
        # break

In [141]:
np.array(covrs).shape

(5000, 55, 55)

In [125]:
# np.array(preds[4000].numpy()).shape

(110,)

In [105]:
# PhotometricSystem.get_available_systems()

'DECam, Els_Custom_W09_S2, Euclid_VIS, Gaia_2, Gaia_DR3_Vega, Halpha_Custom_AB, H_Custom, Hipparcos_Tycho, HST_ACSWFC, HST_HUGS_Std, HST_WFC3UVIS, HST_WFPC2, IPHAS, JKC, JKC_Std, JPAS, JPLUS, JWST_NIRCAM, PanSTARRS1, PanSTARRS1_Std, Pristine, SDSS, SDSS_Std, Sky_Mapper, Stromgren, Stromgren_Std, WFIRST'

In [106]:
phot_system_list = [
    PhotometricSystem.Gaia_2,
    PhotometricSystem.Gaia_DR3_Vega,
    PhotometricSystem.PanSTARRS1,
    PhotometricSystem.PanSTARRS1_Std,
    PhotometricSystem.Pristine,
    PhotometricSystem.SDSS,
    PhotometricSystem.SDSS_Std
]

In [107]:
len(preds[0][0].numpy()[55:])

55

In [143]:
df = pd.DataFrame(
    {'source_id':range(len(preds)),
     'bp_coefficients':[pred.numpy()[:55] for pred in preds],
     'bp_standard_deviation':[np.std(pred.numpy()[:55]) for pred in preds],
     'bp_coefficient_covariances':covbs,
     'rp_coefficients':[pred.numpy()[55:] for pred in preds],
     'rp_coefficient_covariances':covrs,
     'rp_standard_deviation':[np.std(pred.numpy()[55:]) for pred in preds]
    }
)

In [144]:
df

,source_id,bp_coefficients,bp_standard_deviation,bp_coefficient_covariances,rp_coefficients,rp_coefficient_covariances,rp_standard_deviation
0,0,"[-0.66917336, 0.48170173, 0.3844219, -0.878041...",0.395025,"[[0.8794816779873713, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.5746207, 0.38471243, 0.55732507, -0.361483...","[[0.8618410848556302, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.332891
1,1,"[-0.92343724, 0.46212238, 0.6806567, -0.442677...",0.365920,"[[0.006710976393270451, 0.0, 0.0, 0.0, 0.0, 0....","[-0.77701974, 0.72113556, 0.40874174, -0.11885...","[[0.37018470542250625, 0.0, 0.0, 0.0, 0.0, 0.0...",0.392543
2,2,"[0.030043442, -0.09648458, -0.17177546, 0.4975...",0.190099,"[[0.41990639715155353, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.07388496, -0.26487806, -0.22752109, -0.218...","[[0.5307518640938866, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.111023
3,3,"[0.1197419, -0.14983305, -0.32838795, 0.129001...",0.467134,"[[1.2576892887864375, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.03923164, -0.39923033, -0.3175685, -0.3573...","[[1.7363151434227717, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.193433
4,4,"[-0.9323662, 0.43192646, 0.8958967, -0.2838097...",0.365687,"[[0.9400289624509561, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.72170985, 0.917938, 1.2655687, -0.66903114...","[[0.7515430424844141, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.440053
...,...,...,...,...,...,...,...
4995,4995,"[-0.77967006, 0.5356065, 0.44881177, -0.482289...",0.320204,"[[0.30650636868447023, 0.0, 0.0, 0.0, 0.0, 0.0...","[-0.59939426, 0.5576456, 0.027809292, 0.274802...","[[0.017760566005055267, 0.0, 0.0, 0.0, 0.0, 0....",0.200720
4996,4996,"[-0.5080072, 0.26522183, 0.23295112, 0.0683715...",0.294034,"[[0.6448679250979694, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.44611213, 0.22348116, -0.06383711, -0.0586...","[[0.1366999171389125, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.177519
4997,4997,"[0.0064013377, 0.2473522, -0.40573567, 0.16585...",0.287771,"[[0.08349001638185882, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.060025718, -0.0032160357, -0.10103643, 0.26...","[[0.05431244310895722, 0.0, 0.0, 0.0, 0.0, 0.0...",0.122908
4998,4998,"[0.623542, -0.7065894, -0.33187175, 0.8890895,...",0.613836,"[[1.9588125025371852, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.2961528, -0.7852448, -0.5386681, -0.3675081...","[[0.5378055136336037, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.232720


In [110]:
f = '/arc/home/aydanmckay/XpContinuousMeanSpectrum_407725-409897.csv'

In [111]:
# synthetic_photometry = generate(f, photometric_system=phot_system_list)
synthetic_photometry = generate(df, photometric_system=PhotometricSystem.Pristine)
synthetic_photometry

,source_id,Pristine_mag_CaHK,Pristine_flux_CaHK,Pristine_flux_error_CaHK
0,1,21.31737,2.164373e-19,5.092641e-19


In [63]:
from gaiaxpy.core.satellite import BANDS

In [64]:
for band in BANDS:
    print(band)

bp
rp
